In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

In [ ]:
import joblib
import numpy as np
from models.utils import get_artifacts_path
from db.ohlcv.queries import load_ohlcv_cloud
from db.ohlcv.utils import get_ibex_tickers
from models.trees.features import safe_build_features


model_type = "rf"
horizon = 1
name = f"{model_type}_h{horizon}_full.pkl"
artifact = joblib.load(get_artifacts_path() / name)
model = artifact["model"]
feature_cols = artifact["features"]


tickers = get_ibex_tickers()
df_micro = load_ohlcv_cloud(tickers, 50)
print(df_micro.shape)
df_micro.tail(1)

(1500, 7)


,ticker,date,open,high,low,close,volume
1495,UNI.MC,2026-02-12,2.738,2.760,2.642,2.654,5809295
1496,UNI.MC,2026-02-13,2.650,2.672,2.586,2.608,5622333
1497,UNI.MC,2026-02-16,2.626,2.650,2.610,2.618,4424773
1498,UNI.MC,2026-02-17,2.604,2.638,2.598,2.638,4542761
1499,UNI.MC,2026-02-18,2.662,2.714,2.662,2.704,3136601


In [3]:
df = safe_build_features(df_micro, horizon)
remove_cols = [
    "ticker", "date", "open", "high", "low",
    "close", "volume", "target", "future_log_ret"
]
X = df.drop(columns=remove_cols)
X = X[feature_cols]
X = X.replace([np.inf, -np.inf], np.nan)
mask = X.notna().all(axis=1)
X = X.loc[mask]

df_pred = df.loc[mask, ["ticker", "date"]]


In [ ]:
preds = model.predict(X)
proba = model.predict_proba(X)[:, 1]

df_pred["pred"] = preds
df_pred["proba"] = proba

# double check only one prediction for ticker
df_last = df_pred.sort_values("date").groupby("ticker").tail(1)
# df_last = df_last.sort_values("proba", ascending=False)

# select 3 highest confidence (deviation from 0.5)
df_last["confidence"] = abs(df_last["proba"] - 0.5)
df_last = df_last.sort_values("confidence", ascending=False).head(3).reset_index()[["ticker", "pred", "proba"]]



In [15]:
df_last

,ticker,pred,proba
0,MTS.MC,0,0.474259
1,ACS.MC,0,0.487698
2,IBE.MC,1,0.511317


In [6]:
df_pred.shape

(30, 4)